# Data Exploration - Constrained Gait Analysis

This notebook explores the T5 trial data to understand the multi-modal sensor characteristics and constrained gait patterns.

## Objective
- Understand data structure and sampling rates
- Visualize constrained gait patterns (left leg locked in extension)
- Identify key signals for ground truth annotation

In [ ]:
# Setup and imports
import sys
import os
sys.path.append('../src')

# Add the absolute path as backup
current_dir = os.path.dirname(os.path.abspath('__file__'))
src_path = os.path.join(current_dir, '..', 'src')
sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_loader import GaitDataLoader
from synchronizer import MultiModalSynchronizer
from visualizer import GaitDataVisualizer

# Configure plotting
plt.style.use('default')  # Use default style instead of seaborn-v0_8
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Environment setup complete!")
print("Available classes:")
print("- GaitDataLoader")
print("- MultiModalSynchronizer") 
print("- GaitDataVisualizer")

## 1. Load Raw Data

In [ ]:
# Initialize data loader and load T5 trial data
loader = GaitDataLoader(data_dir="../data")

# Load T5 trial data
trial_id = "T5"

print(f"Loading trial {trial_id}...")

# Load individual modalities
kinetics_data = loader.load_kinetics(trial_id)
emg_data = loader.load_emg(trial_id)
kinematics_data = loader.load_kinematics(trial_id)

# Create a dictionary to match expected format
raw_data = {
    'kinetics': kinetics_data,
    'emg': emg_data,
    'kinematics': kinematics_data
}

print(f"Loaded data for trial {trial_id}")
print("\nData modalities:")
for modality, df in raw_data.items():
    print(f"  {modality}: {len(df)} samples, {df['time'].max():.1f}s duration")
    print(f"    Columns: {list(df.columns[:10])}{'...' if len(df.columns) > 10 else ''}")
    print()

## 2. Data Synchronization

In [ ]:
# Data Synchronization using MultiModalSynchronizer
synchronizer = MultiModalSynchronizer(target_rate=1000)

# For now, we'll extract a 20-second window and use kinetics as the base timeline
time_window = 20.0
kinetics_mask = raw_data['kinetics']['time'] <= time_window

# Extract synchronized data for analysis
synchronized_data = {
    'kinetics': raw_data['kinetics'][kinetics_mask].copy(),
    'emg': raw_data['emg'][raw_data['emg']['time'] <= time_window].copy(),
    'kinematics': raw_data['kinematics'][raw_data['kinematics']['time'] <= time_window].copy()
}

print("Data synchronized to analysis window")
print(f"Analysis window: 0 to {time_window} seconds")
print("\nSynchronized data:")
for modality, df in synchronized_data.items():
    sampling_rate = len(df) / df['time'].max() if df['time'].max() > 0 else 0
    print(f"  {modality}: {len(df)} samples, {df['time'].max():.1f}s duration")
    print(f"    Sampling rate: {sampling_rate:.0f} Hz")
    print()

## 3. Force Plate Analysis

In [ ]:
# Analyze force plate data using correct column names
kinetics = synchronized_data['kinetics']

# Plot force plates
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Full duration view (up to 20 seconds)
time = kinetics['time']
axes[0].plot(time, kinetics['Fz_L'], label='Left Force Plate', color='blue', alpha=0.7)
axes[0].plot(time, kinetics['Fz_R'], label='Right Force Plate', color='red', alpha=0.7)
axes[0].set_ylabel('Vertical Force (N)')
axes[0].set_title('Force Plates - 20 Second Analysis Window')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# First 10 seconds (closer look)
mask_10s = time <= 10
axes[1].plot(time[mask_10s], kinetics['Fz_L'][mask_10s], label='Left Force Plate', color='blue', alpha=0.7)
axes[1].plot(time[mask_10s], kinetics['Fz_R'][mask_10s], label='Right Force Plate', color='red', alpha=0.7)
axes[1].set_ylabel('Vertical Force (N)')
axes[1].set_xlabel('Time (seconds)')
axes[1].set_title('Force Plates - First 10 Seconds (Detailed View)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Basic statistics
print("Force Plate Statistics (20-second window):")
print(f"Left plate - Max: {kinetics['Fz_L'].max():.1f}N, Min: {kinetics['Fz_L'].min():.1f}N, Mean: {kinetics['Fz_L'].mean():.1f}N")
print(f"Right plate - Max: {kinetics['Fz_R'].max():.1f}N, Min: {kinetics['Fz_R'].min():.1f}N, Mean: {kinetics['Fz_R'].mean():.1f}N")

# Contact analysis
left_contact = (kinetics['Fz_L'].abs() > 100).sum()
right_contact = (kinetics['Fz_R'].abs() > 100).sum()
print(f"\nContact Analysis:")
print(f"Left contact samples: {left_contact} ({100*left_contact/len(kinetics):.1f}%)")
print(f"Right contact samples: {right_contact} ({100*right_contact/len(kinetics):.1f}%)")
print(f"Total contact: {100*(left_contact + right_contact)/(2*len(kinetics)):.1f}% of stride")

## 4. Kinematic Marker Analysis

In [ ]:
# Analyze available kinematic markers
# Extract 20-second window for kinematics as well
mask_kin = kinematics_data['time'] <= time_window
kinematics_20s = kinematics_data[mask_kin].copy()

# Identify available markers
print("Available kinematic markers:")
marker_cols = [col for col in kinematics_20s.columns if col not in ['Frame', 'Sub Frame', 'time']]
print(f"Total data columns: {len(marker_cols)}")

# Show first 10 columns to understand naming structure
print("\nFirst 10 kinematic columns:")
for i, col in enumerate(marker_cols[:10]):
    print(f"  {i+1}. {col}")

if len(marker_cols) > 10:
    print(f"  ... and {len(marker_cols)-10} more columns")

# Plot first few kinematic signals (first 20 seconds)
fig, ax = plt.subplots(figsize=(15, 6))

time_20s = kinematics_20s['time']

# Plot first 4 non-frame/time columns
plot_cols = marker_cols[:4]
colors = plt.cm.tab10(np.linspace(0, 1, len(plot_cols)))

for i, col in enumerate(plot_cols):
    if col in kinematics_20s.columns:
        ax.plot(time_20s, kinematics_20s[col], 
               label=col, color=colors[i], alpha=0.8)

ax.set_ylabel('Position/Angle (units vary)')
ax.set_xlabel('Time (seconds)')
ax.set_title('Sample Kinematic Signals (First 4 Channels)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nKinematics data summary:")
print(f"Duration: {kinematics_20s['time'].max():.1f} seconds")
print(f"Samples: {len(kinematics_20s)}")
print(f"Estimated sampling rate: {len(kinematics_20s) / kinematics_20s['time'].max():.0f} Hz")

## 5. EMG Analysis

In [ ]:
# Analyze EMG data
# Extract 20-second window for EMG
mask_emg = emg_data['time'] <= time_window
emg_20s = emg_data[mask_emg].copy()

print(f"EMG data loaded for analysis")
print(f"Duration: {emg_20s['time'].max():.1f} seconds")
print(f"Samples: {len(emg_20s)}")
print(f"Estimated sampling rate: {len(emg_20s) / emg_20s['time'].max():.0f} Hz")

# Get EMG channels
emg_cols = [col for col in emg_20s.columns if 'IM EMG' in col]
print(f"\nAvailable EMG channels: {len(emg_cols)}")

# Simple envelope computation (moving RMS)
def compute_rms_envelope(signal, window_size=100):
    """Compute RMS envelope using rolling window."""
    return signal.rolling(window=window_size, center=True).apply(lambda x: np.sqrt(np.mean(x**2)))

# Plot first 4 EMG channels with envelopes (first 10 seconds for clarity)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

mask_10s = emg_20s['time'] <= 10
time_10s = emg_20s['time'][mask_10s]

for i, col in enumerate(emg_cols[:4]):
    if i < 4:
        # Raw EMG
        raw_signal = emg_20s[col][mask_10s]
        envelope = compute_rms_envelope(raw_signal, window_size=50)
        
        axes[i].plot(time_10s, raw_signal, alpha=0.3, color='gray', label='Raw EMG')
        axes[i].plot(time_10s, envelope, color='red', linewidth=2, label='RMS Envelope')
        axes[i].set_title(f'{col}')
        axes[i].set_ylabel('Amplitude (V)')
        axes[i].grid(True, alpha=0.3)
        axes[i].legend()
        
        if i >= 2:  # Bottom row
            axes[i].set_xlabel('Time (seconds)')

plt.suptitle('EMG Signals with RMS Envelopes (First 10 seconds)')
plt.tight_layout()
plt.show()

print("\nEMG channels available:")
for i, col in enumerate(emg_cols[:8]):  # Show first 8
    print(f"  {i+1}. {col}")
if len(emg_cols) > 8:
    print(f"  ... and {len(emg_cols)-8} more channels")

## 6. Constrained Gait Pattern Analysis

In [ ]:
# Constrained Gait Pattern Analysis using GaitDataVisualizer
visualizer = GaitDataVisualizer(figsize=(15, 12))

# Create comprehensive constrained gait visualization
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Get kinetics data for plotting
kinetics = synchronized_data['kinetics']
time = kinetics['time']

# 1. Force plate comparison
axes[0].plot(time, -kinetics['Fz_L'], label='Left Force Plate', color='blue', alpha=0.7, linewidth=1)
axes[0].plot(time, -kinetics['Fz_R'], label='Right Force Plate', color='red', alpha=0.7, linewidth=1)
axes[0].axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='Contact Threshold')
axes[0].set_ylabel('Vertical Force (N)')
axes[0].set_title('Constrained Gait Analysis - Force Patterns')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Force asymmetry over time (5-second sliding window)
window_size = 5000  # 5 seconds at 1000Hz
left_force_abs = kinetics['Fz_L'].abs()
right_force_abs = kinetics['Fz_R'].abs()

# Calculate sliding window asymmetry
asymmetry_ratio = []
time_centers = []

for i in range(window_size//2, len(kinetics) - window_size//2, 1000):  # Every second
    left_sum = left_force_abs.iloc[i-window_size//2:i+window_size//2].sum()
    right_sum = right_force_abs.iloc[i-window_size//2:i+window_size//2].sum()
    total = left_sum + right_sum
    if total > 0:
        asymmetry = right_sum / total  # Right leg proportion
        asymmetry_ratio.append(asymmetry)
        time_centers.append(kinetics['time'].iloc[i])

axes[1].plot(time_centers, asymmetry_ratio, 'g-', marker='o', linewidth=2, markersize=4)
axes[1].axhline(y=0.5, color='gray', linestyle='-', alpha=0.5, label='Equal Loading')
axes[1].axhline(y=0.7, color='orange', linestyle='--', alpha=0.7, label='Expected Constraint (~70%)')
axes[1].set_ylabel('Right Leg Loading Ratio')
axes[1].set_title('Loading Asymmetry Over Time (5-second sliding window)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1)

# 3. Contact phases
contact_threshold = 100
left_contact = (left_force_abs > contact_threshold).astype(int)
right_contact = (right_force_abs > contact_threshold).astype(int)

axes[2].fill_between(time, 0, left_contact, alpha=0.6, color='blue', label='Left Contact')
axes[2].fill_between(time, 1, 1 + right_contact, alpha=0.6, color='red', label='Right Contact')
axes[2].set_ylabel('Contact State')
axes[2].set_xlabel('Time (seconds)')
axes[2].set_title('Gait Contact Phases')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim(-0.1, 2.1)
axes[2].set_yticks([0.5, 1.5])
axes[2].set_yticklabels(['Left', 'Right'])

plt.tight_layout()
plt.show()

# Quantify asymmetry
mask_analysis = time <= 20
left_force_analysis = kinetics['Fz_L'][mask_analysis].abs()
right_force_analysis = kinetics['Fz_R'][mask_analysis].abs()

# Calculate overall asymmetry metrics
left_total = left_force_analysis.sum()
right_total = right_force_analysis.sum()
asymmetry_ratio_overall = right_total / (left_total + right_total)

print("\nConstrained Gait Analysis (20-second window):")
print(f"Left leg total loading: {left_total:.0f} N·s")
print(f"Right leg total loading: {right_total:.0f} N·s")
print(f"Right leg bias: {asymmetry_ratio_overall:.1%}")
print(f"Expected with left leg constraint: ~60-70% right bias")

# Contact time analysis
left_contact_time = left_contact[mask_analysis].sum() / 1000  # Convert to seconds
right_contact_time = right_contact[mask_analysis].sum() / 1000
total_analysis_time = 20.0

print(f"\nContact Time Analysis:")
print(f"Left contact time: {left_contact_time:.1f}s ({100*left_contact_time/total_analysis_time:.1f}%)")
print(f"Right contact time: {right_contact_time:.1f}s ({100*right_contact_time/total_analysis_time:.1f}%)")
print(f"Double support: {min(left_contact_time, right_contact_time):.1f}s estimated")

## 7. Data Quality Assessment

In [ ]:
# Data Quality Assessment for Ground Truth Annotation
print("Data Quality Assessment for Ground Truth Annotation")
print("="*60)

# Check for missing data in each modality
datasets = {
    'kinetics': synchronized_data['kinetics'],
    'emg': synchronized_data['emg'],
    'kinematics': synchronized_data['kinematics']
}

for name, df in datasets.items():
    missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    print(f"{name.capitalize():12} - Missing data: {missing_pct:.2f}%")

print()

# Signal quality indicators for kinetics (main annotation signal)
kinetics_analysis = synchronized_data['kinetics']

# Force plate signal-to-noise ratio estimation
# Use first 1 second as baseline noise estimate
baseline_samples = 1000  # First second
left_baseline_std = kinetics_analysis['Fz_L'][:baseline_samples].std()
right_baseline_std = kinetics_analysis['Fz_R'][:baseline_samples].std()

left_signal_std = kinetics_analysis['Fz_L'].std()
right_signal_std = kinetics_analysis['Fz_R'].std()

left_snr = left_signal_std / left_baseline_std if left_baseline_std > 0 else 0
right_snr = right_signal_std / right_baseline_std if right_baseline_std > 0 else 0

print(f"Force plate signal quality:")
print(f"  Left plate SNR estimate:  {left_snr:.1f}")
print(f"  Right plate SNR estimate: {right_snr:.1f}")
print(f"  Status: {'✓ Good' if min(left_snr, right_snr) > 3 else '⚠ Check quality'}")

print()

# Estimate number of gait cycles using simple peak detection
from scipy.signal import find_peaks

# Find peaks in right force plate (more active leg)
right_force_positive = -kinetics_analysis['Fz_R']  # Convert to positive for peak detection
peaks, properties = find_peaks(right_force_positive, 
                              height=200,     # Minimum peak height
                              distance=300,   # Minimum 0.3s between peaks
                              prominence=100) # Peak prominence

estimated_cycles = len(peaks)
estimated_events = estimated_cycles * 4  # 4 events per cycle (2 legs × heel strike + toe off)

print(f"Gait event estimation (using right leg peaks):")
print(f"  Detected peaks: {len(peaks)}")
print(f"  Estimated gait cycles: {estimated_cycles}")
print(f"  Expected total events: {estimated_events}")
print(f"  Estimated cadence: {estimated_cycles / 20 * 60:.1f} steps/min")

# Show peak times
if len(peaks) > 0:
    peak_times = kinetics_analysis['time'].iloc[peaks].values
    print(f"  Peak times: {peak_times[:5].round(1).tolist()}{'...' if len(peaks) > 5 else ''}")

print()

# Get asymmetry from previous analysis
kinetics = synchronized_data['kinetics']
time = kinetics['time']
left_force_analysis = kinetics['Fz_L'].abs()
right_force_analysis = kinetics['Fz_R'].abs()
left_total = left_force_analysis.sum()
right_total = right_force_analysis.sum()
asymmetry_ratio_overall = right_total / (left_total + right_total)

# Contact time analysis
contact_threshold = 100
left_contact = (left_force_analysis > contact_threshold).astype(int)
right_contact = (right_force_analysis > contact_threshold).astype(int)
left_contact_time = left_contact.sum() / 1000  # Convert to seconds
right_contact_time = right_contact.sum() / 1000

# Data suitability assessment
suitability_checks = {
    'Data completeness': missing_pct < 1.0,
    'Signal quality': min(left_snr, right_snr) > 3,
    'Gait cycles detected': estimated_cycles > 10,
    'Force asymmetry visible': abs(asymmetry_ratio_overall - 0.5) > 0.1,
    'Contact phases clear': (left_contact_time > 5) and (right_contact_time > 5)
}

print("Annotation Suitability Checklist:")
for check, passed in suitability_checks.items():
    status = "✓ Pass" if passed else "✗ Fail"
    print(f"  {check}: {status}")

overall_suitable = all(suitability_checks.values())
print(f"\nOverall Assessment: {'✓ SUITABLE' if overall_suitable else '⚠ NEEDS REVIEW'}")

if overall_suitable:
    print(f"Recommendation: Proceed with manual annotation")
    print(f"Target: Annotate ~{estimated_events} gait events over 20-second window")
    print(f"Priority: Focus on heel strike and toe off events for both legs")
else:
    print("Recommendation: Review data quality issues before annotation")

print()
print("Next Steps:")
print("1. Use notebook 02_annotation_tool.ipynb for interactive annotation")
print("2. Focus on clear force plate transitions for heel strike/toe off detection")
print("3. Use kinematic data for validation of ambiguous events")
print("4. Validate annotations with notebook 03_validation.ipynb")

## Summary

This exploration confirms:

1. **Multi-modal data is properly synchronized** to 1000 Hz timeline
2. **Clear constrained gait patterns** with right leg compensation
3. **Good signal quality** across all modalities
4. **Identifiable gait events** in force and kinematic data
5. **Suitable for manual annotation** with expected ~40-50 events

**Next Step**: Proceed to interactive annotation in notebook `02_annotation_tool.ipynb`